In [17]:
# Install dependencies (run once in Colab)
!pip install yfinance numpy pandas scipy matplotlib ipywidgets --quiet

# Imports
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.stats import chi2
import matplotlib.pyplot as plt
from scipy.linalg import sqrtm
import ipywidgets as widgets
from IPython.display import display, clear_output

# ------------------ Helper functions ------------------

def download_data(ticker1, ticker2, start, end):
    """
    Download adjusted close prices for two tickers from Yahoo Finance.
    """
    tickers = [ticker1.strip().upper(), ticker2.strip().upper()]
    df = yf.download(tickers, start=start, end=end, progress=False)['Close']
    if isinstance(df, pd.Series):
        raise ValueError("Only one ticker was retrieved; ensure both ticker symbols are valid.")
    df = df.dropna()
    if df.shape[0] < 2:
        raise ValueError("Not enough data in the given date range.")
    return df

def compute_log_returns(prices):
    """
    Compute daily log returns from price series.
    """
    return np.log(prices / prices.shift(1)).dropna()

def scale_to_horizon(mean_daily, cov_daily, T):
    """
    Scale daily mean and covariance to horizon T assuming independence.
    """
    mean_T = mean_daily * T
    cov_T = cov_daily * T
    return mean_T, cov_T

def compute_region_parameters(last_prices, mean_T, cov_T, p):
    """
    Compute parameters of the confidence region G in log-price space.
    Returns mu_log (expected log-price), Sigma (covariance), and threshold c.
    """
    log_S0 = np.log(last_prices.values)
    mu_log = log_S0 + mean_T.values    # expected future log-price (median center)
    Sigma = cov_T.values               # covariance over horizon
    c = chi2.ppf(p, df=2)              # chi-square threshold for 2 dimensions
    return mu_log, Sigma, c

def is_in_region(S_vec, mu_log, Sigma, c, regularize_eps=1e-10):
    """
    Test if candidate price vector S_vec = [S1, S2] lies inside G.
    Returns (bool, statistic).
    """
    log_S = np.log(np.array(S_vec))
    delta = log_S - mu_log
    # regularize if needed
    Sigma_reg = Sigma.copy()
    try:
        inv = np.linalg.inv(Sigma_reg)
    except np.linalg.LinAlgError:
        Sigma_reg += np.eye(2) * regularize_eps
        inv = np.linalg.inv(Sigma_reg)
    stat = delta.T @ inv @ delta
    return stat <= c, stat

def plot_confidence_ellipse(mu_log, Sigma, c, p):
    """
    Plot the confidence region G in price space by exponentiating
    the ellipse defined in log-space.
    """
    theta = np.linspace(0, 2 * np.pi, 300)
    unit_circle = np.vstack([np.cos(theta), np.sin(theta)])  # 2 x N

    L = sqrtm(Sigma * c)
    ellipse_log = mu_log[:, None] + L @ unit_circle  # 2 x N in log-space
    ellipse_price = np.exp(ellipse_log)
    center_price = np.exp(mu_log)

    plt.figure(figsize=(6,6))
    plt.plot(ellipse_price[0, :], ellipse_price[1, :], label=f"{int(100*p)}% confidence region")
    plt.scatter([center_price[0]], [center_price[1]], marker='x', label="Median (center)", zorder=3)
    plt.xlabel("S1^T")
    plt.ylabel("S2^T")
    plt.title("Region G in Price Space (Ellipse)")
    plt.legend()
    plt.grid(True)
    plt.axis("equal")
    plt.tight_layout()
    plt.show()

def empirical_check(mu_log, Sigma, c, n_samples=50000, seed=0):
    """
    Monte Carlo empirical estimate of P((S1^T, S2^T) ∈ G), assuming
    log S^T ~ N(mu_log, Sigma).
    """
    rng = np.random.default_rng(seed)
    samples_log = rng.multivariate_normal(mean=mu_log, cov=Sigma, size=n_samples)
    deltas = samples_log - mu_log
    inv = np.linalg.inv(Sigma)
    stats = np.einsum('ij,jk,ik->i', deltas, inv, deltas)
    return np.mean(stats <= c)

def area_log_space(Sigma, c):
    """
    Exact area of the confidence ellipse in log-price space.
    """
    return np.pi * c * np.sqrt(np.linalg.det(Sigma))

def estimate_price_space_area(mu_log, Sigma, c, n_samples=50000, seed=0):
    """
    Monte Carlo estimate of the area of G in price space:
    Area_price = ∫_{x in ellipse_log} exp(x1 + x2) dx
    We sample uniformly in the log-space ellipse and weight by exp(x1+x2).
    """
    rng = np.random.default_rng(seed)
    # Map unit disk to ellipse: x = mu_log + L @ u, where L = sqrt(c * Sigma)
    L = sqrtm(c * Sigma)  # 2x2
    # Uniform sampling in unit disk via polar (r = sqrt(u))
    u = rng.random(n_samples)
    r = np.sqrt(u)
    theta = rng.random(n_samples) * 2 * np.pi
    unit_disk = np.vstack([r * np.cos(theta), r * np.sin(theta)])  # 2 x N
    xs_log = mu_log[:, None] + L @ unit_disk  # 2 x N
    weights = np.exp(xs_log[0, :] + xs_log[1, :])  # Jacobian factor for exp transform
    avg_weight = np.mean(weights)
    area_log = area_log_space(Sigma, c)
    area_price = area_log * avg_weight
    return area_price

# ------------------ Widget setup ------------------

# Primary inputs
ticker1_w = widgets.Text(value="AAPL", description="Ticker 1:", continuous_update=False)
ticker2_w = widgets.Text(value="MSFT", description="Ticker 2:", continuous_update=False)
start_w = widgets.Text(value="2023-01-01", description="Start (YYYY-MM-DD):", continuous_update=False)
end_w = widgets.Text(value="2025-07-31", description="End (YYYY-MM-DD):", continuous_update=False)
T_w = widgets.IntSlider(value=20, min=1, max=252, step=1, description="Horizon T:", continuous_update=False)
p_w = widgets.FloatSlider(value=0.95, min=0.5, max=0.999, step=0.001, description="Confidence p:", readout_format=".3f", continuous_update=False)
mc_samples_w = widgets.IntSlider(value=30000, min=1000, max=100000, step=1000, description="MC samples:", continuous_update=False)
plot_checkbox = widgets.Checkbox(value=True, description="Show ellipse plot")
run_button = widgets.Button(description="Run analysis", button_style="primary")

# Test pair multipliers (S1/S2 factor)
test_multiplier1 = widgets.FloatText(value=1.0, description="S1 factor:", step=0.01)
test_multiplier2 = widgets.FloatText(value=1.0, description="S2 factor:", step=0.01)

# Output area
out = widgets.Output()

# Callback
def on_run_clicked(b):
    with out:
        clear_output()
        try:
            # Read inputs
            t1 = ticker1_w.value.strip().upper()
            t2 = ticker2_w.value.strip().upper()
            start = start_w.value.strip()
            end = end_w.value.strip()
            T = float(T_w.value)
            p = float(p_w.value)
            mc_samples = int(mc_samples_w.value)
            do_plot = plot_checkbox.value
            f1 = float(test_multiplier1.value)
            f2 = float(test_multiplier2.value)

            # Download historical data
            df_prices = download_data(t1, t2, start, end)
            log_returns = compute_log_returns(df_prices)
            mean_daily = log_returns.mean()
            cov_daily = log_returns.cov()

            # Scale to horizon
            mean_T, cov_T = scale_to_horizon(mean_daily, cov_daily, T)
            last_prices = df_prices.iloc[-1]

            # Region parameters
            mu_log, Sigma, c = compute_region_parameters(last_prices, mean_T, cov_T, p)
            median_price = np.exp(mu_log)

            # Display summary
            print("=== Results ===")
            print(f"Tickers: {t1}, {t2}")
            print(f"Date window: {start} to {end}")
            print(f"Horizon T (trading days): {T}")
            print(f"Confidence level p: {p}")
            print("\nLast observed prices (S0):")
            display(last_prices.to_frame(name="Price"))
            print("\nDaily log-return mean:")
            display(mean_daily.to_frame(name="Mean"))
            print("Daily covariance matrix:")
            display(cov_daily)

            print(f"\nScaled mean over horizon T={T} (log returns):")
            display(mean_T.to_frame(name="Mean_T"))
            print(f"Scaled covariance over horizon T={T}:")
            display(cov_T)

            print(f"\nRegion G parameters:")
            print(f"  c (chi-square threshold for p={p:.3f}): {c:.6f}")
            print(f"  mu_log (expected log-price): {mu_log}")
            print(f"  Median price (exp(mu_log)):\n{median_price}")

            # Check median inside G
            inside_median, stat_median = is_in_region(median_price, mu_log, Sigma, c)
            print(f"\nCheck: is the median price inside G? {inside_median} (statistic = {stat_median:.4f})")

            # Exact log-space area
            area_log = area_log_space(Sigma, c)
            print(f"\nExact area in log-price space: {area_log:.6e}")

            # Estimate price-space area
            area_price_est = estimate_price_space_area(mu_log, Sigma, c, n_samples=mc_samples)
            print(f"Estimated area in price space (MC samples={mc_samples}): {area_price_est:.6e}")

            # Empirical probability via Monte Carlo
            empirical_p = empirical_check(mu_log, Sigma, c, n_samples=mc_samples)
            print(f"\nEmpirical estimate P((S1^T,S2^T) ∈ G): {empirical_p:.4f}")

            # Test arbitrary pair via factors
            test_pair = [median_price[0] * f1, median_price[1] * f2]
            inside_test, stat_test = is_in_region(test_pair, mu_log, Sigma, c)
            print(f"\nTest pair {test_pair} (factors {f1}, {f2}): inside G? {inside_test} (statistic = {stat_test:.4f})")

            # Optional plot
            if do_plot:
                plot_confidence_ellipse(mu_log, Sigma, c, p)

        except Exception as e:
            print("Error during computation:", str(e))

run_button.on_click(on_run_clicked)

# Layout
inputs_row1 = widgets.HBox([ticker1_w, ticker2_w, start_w, end_w])
inputs_row2 = widgets.HBox([T_w, p_w, mc_samples_w])
test_row = widgets.HBox([test_multiplier1, test_multiplier2, plot_checkbox])
controls = widgets.VBox([inputs_row1, inputs_row2, test_row, run_button])
display(controls, out)


Output()